In [44]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import geopy.distance
from math import radians, cos, sin, asin, sqrt
import pickle
import json

In [124]:
cities_df

,city,population,lat,lng,resources
0,Belgrade,1378682.0,44.8167,20.4667,2757.364
1,Novi Sad,380000.0,45.2644,19.8317,760.000
2,Niš,183164.0,43.3192,21.8961,366.328
3,Zemun,161596.0,44.8500,20.4000,323.192
4,Kragujevac,150623.0,44.0142,20.9394,301.246
5,Subotica,105681.0,46.0983,19.6700,211.362
6,Valjevo,90312.0,44.2667,19.8833,180.624
7,Loznica,86413.0,44.5333,19.2258,172.826
8,Zrenjanin,76511.0,45.3778,20.3861,153.022
9,Pančevo,76203.0,44.8739,20.6519,152.406


In [203]:
def preprocess_distances(cities, available_res, path='distances.pkl'):
    dist = np.zeros((n,k))
    for i in range(n):
        for j in range(k):
            dist[i,j] = geopy.distance.distance(cities.iloc[i][['lat','lng']].values, 
                                                available_res.iloc[j][['lat','lng']].values).km
    with open(path, 'wb') as out:
        pickle.dump(dist, out, pickle.HIGHEST_PROTOCOL)
    
def load_distances(path='distances.pkl'):
    with open('distances.pkl', 'rb') as inp:
        return pickle.load(inp)

def estimate_endangered(cities, earthquakes):
    endangered = []
    for i in range(len(cities)):
        curr = 0
        for eq in earthquakes:
            dist = geopy.distance.distance(cities.iloc[i][['lat','lng']].values, 
                                                eq['epicenter']).km
            m = eq['magnitude']
            base = 60000*np.exp((m/5)-1)/(np.exp(2)-1)/(1+(dist**4))
            curr = max(curr, base * cities.iloc[i]['population'])
        endangered.append(curr)
    return np.array(endangered)

In [208]:
# Prepare data
earthquakes = [
    {'epicenter':(44.117512, 20.189369), 'magnitude': 6},
    {'epicenter':(43.886935, 20.556935), 'magnitude': 8.5},
    {'epicenter':(43.654546, 20.838094), 'magnitude': 8},
    {'epicenter':(45.415727, 20.205249), 'magnitude': 8}
]

cities_df = pd.read_csv('serbia_cities.csv')[['city', 'population', 'lat', 'lng']].iloc[0:17]
n = len(cities_df)
np.random.seed(0)
endangered = estimate_endangered(cities_df, earthquakes)
print("Endangered")

goal = endangered / 10

available_res = pd.DataFrame(cities_df)#.iloc[0:10]
k = len(available_res)
available_res['resources'] = available_res['population'] / 300
available_res_df = available_res.drop(['population'], axis=1)
resources = available_res_df['resources'].astype(np.int64)

preprocess_distances(cities_df, available_res)
dist = load_distances()

pd.DataFrame([cities_df.city, goal, resources]).T.sort_values(by='Unnamed 0', ascending=False)

Endangered


,city,Unnamed 0,resources
8,Zrenjanin,2747.373566,255
11,Kraljevo,2665.155926,222
10,Čačak,1764.653948,239
1,Novi Sad,500.273574,1266
4,Kragujevac,218.208583,502
6,Valjevo,135.752031,301
0,Belgrade,100.082789,4595
15,Kruševac,38.392764,195
14,Užice,25.95904,199
3,Zemun,15.768942,538


In [205]:
%%time
from scipy.optimize import minimize

# Utilities
def toVector(x): # n x k
    assert x.shape == (n, k)
    return x.flatten()

def toMat(vec):
    assert vec.shape == (n*k,)
    return vec.reshape(n,k)

# Cost function
alpha = 0.1
def cost_fun(x):
    x = toMat(x)
    cost1 = np.sum((np.sum(x, axis=1) - goal)**2)
    cost2 = np.sum(dist * x)
    return cost1 + alpha*cost2

# Constraints: k
def get_constraint(j):
    def constraint(x):
        x = toMat(x)
        return resources[j] - np.sum(x[:,j])
    return constraint

constraints = [{'type': 'ineq', 'fun':get_constraint(j)} 
                  for j in range(k)]

# Bounds
bnds = []
for i in range(n):
    for j in range(k):
        bnds.append((0, resources[j]))
print('Bounds:', len(bnds))

# initial guess
x0 = np.zeros((n, k))
#x0[0] = available_res

# show initial objective
print('Initial Objective: ' + str(cost_fun(toVector(x0))))

# optimize
solution = minimize(cost_fun, toVector(x0), bounds=(bnds), constraints=constraints)
x = np.array(solution.x).astype(np.int64)


# show final objective
print('Final Objective: ' + str(cost_fun(x)))

print('Used resources:', sum(x))

# print solution
print('Solution')
#print(x)

Bounds: 289
Initial Objective: 18093973.019058477
Final Objective: 52215.282655715986
Used resources: 8187
Solution
CPU times: user 22.4 s, sys: 148 ms, total: 22.5 s
Wall time: 22.7 s


In [211]:
print(goal)
print(sum(goal))

[1.00082789e+02 5.00273574e+02 4.12851915e+00 1.57689424e+01
 2.18208583e+02 3.22376341e+00 1.35752031e+02 1.53259161e+00
 2.74737357e+03 5.52092044e+00 1.76465395e+03 2.66515593e+03
 7.80115538e+00 5.74981665e-01 2.59590400e+01 3.83927638e+01
 1.88454714e-01]
8234.591550048342


In [213]:
sum(resources)

10420

In [ ]:
final_dist = pd.DataFrame(toMat(x).T)
final_dist.columns = cities_df['city']
final_dist.index = cities_df['city']
final_dist

In [214]:
print(final_dist.T.to_json())

{"Belgrade":{"Belgrade":100,"Novi Sad":0,"Ni\u0161":0,"Zemun":0,"Kragujevac":0,"Subotica":0,"Valjevo":0,"Loznica":0,"Zrenjanin":951,"Pan\u010devo":0,"\u010ca\u010dak":1176,"Kraljevo":1166,"Novi Pazar":0,"Leskovac":0,"U\u017eice":0,"Kru\u0161evac":0,"Vranje":0},"Novi Sad":{"Belgrade":0,"Novi Sad":499,"Ni\u0161":0,"Zemun":0,"Kragujevac":0,"Subotica":0,"Valjevo":0,"Loznica":0,"Zrenjanin":766,"Pan\u010devo":0,"\u010ca\u010dak":0,"Kraljevo":0,"Novi Pazar":0,"Leskovac":0,"U\u017eice":0,"Kru\u0161evac":0,"Vranje":0},"Ni\u0161":{"Belgrade":0,"Novi Sad":0,"Ni\u0161":3,"Zemun":0,"Kragujevac":0,"Subotica":0,"Valjevo":0,"Loznica":0,"Zrenjanin":0,"Pan\u010devo":0,"\u010ca\u010dak":0,"Kraljevo":606,"Novi Pazar":0,"Leskovac":0,"U\u017eice":0,"Kru\u0161evac":0,"Vranje":0},"Zemun":{"Belgrade":0,"Novi Sad":0,"Ni\u0161":0,"Zemun":15,"Kragujevac":0,"Subotica":0,"Valjevo":0,"Loznica":0,"Zrenjanin":522,"Pan\u010devo":0,"\u010ca\u010dak":0,"Kraljevo":0,"Novi Pazar":0,"Leskovac":0,"U\u017eice":0,"Kru\u0161eva

city,Belgrade,Novi Sad,Niš,Zemun,Kragujevac,Subotica,Valjevo,Loznica,Zrenjanin,Pančevo,Čačak,Kraljevo,Novi Pazar,Leskovac,Užice,Kruševac,Vranje
city,,,,,,,,,,,,,,,,,
Belgrade,100,0,0,0,0,0,0,0,951,0,1176,1166,0,0,0,0,0
Novi Sad,0,499,0,0,0,0,0,0,766,0,0,0,0,0,0,0,0
Niš,0,0,3,0,0,0,0,0,0,0,0,606,0,0,0,0,0
Zemun,0,0,0,15,0,0,0,0,522,0,0,0,0,0,0,0,0
Kragujevac,0,0,0,0,213,0,0,0,0,0,0,288,0,0,0,0,0
Subotica,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
Valjevo,0,0,0,0,0,0,133,0,0,0,167,0,0,0,0,0,0
Loznica,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
Zrenjanin,0,0,0,0,0,0,0,0,254,0,0,0,0,0,0,0,0


In [230]:
available_res[['city','resources']].to_json()

'{"city":{"0":"Belgrade","1":"Novi Sad","2":"Ni\\u0161","3":"Zemun","4":"Kragujevac","5":"Subotica","6":"Valjevo","7":"Loznica","8":"Zrenjanin","9":"Pan\\u010devo","10":"\\u010ca\\u010dak","11":"Kraljevo","12":"Novi Pazar","13":"Leskovac","14":"U\\u017eice","15":"Kru\\u0161evac","16":"Vranje"},"resources":{"0":4595.6066666667,"1":1266.6666666667,"2":610.5466666667,"3":538.6533333333,"4":502.0766666667,"5":352.27,"6":301.04,"7":288.0433333333,"8":255.0366666667,"9":254.01,"10":239.61,"11":222.2933333333,"12":221.7566666667,"13":200.96,"14":199.1566666667,"15":195.8166666667,"16":183.7933333333}}'

In [228]:
endangered = endangered.astype(np.int64)

In [229]:
ugrozeni = pd.DataFrame(np.array([cities_df.city, endangered]).T)
ugrozeni.columns = ['city', 'ugrozenih']
ugrozeni = ugrozeni.sort_values(by='ugrozenih', ascending=False)
ugrozeni.to_json()

'{"city":{"8":"Zrenjanin","11":"Kraljevo","10":"\\u010ca\\u010dak","1":"Novi Sad","4":"Kragujevac","6":"Valjevo","0":"Belgrade","15":"Kru\\u0161evac","14":"U\\u017eice","3":"Zemun","12":"Novi Pazar","9":"Pan\\u010devo","2":"Ni\\u0161","5":"Subotica","7":"Loznica","13":"Leskovac","16":"Vranje"},"ugrozenih":{"8":27473,"11":26651,"10":17646,"1":5002,"4":2182,"6":1357,"0":1000,"15":383,"14":259,"3":157,"12":78,"9":55,"2":41,"5":32,"7":15,"13":5,"16":1}}'

In [231]:
print(json.dumps(earthquakes))

[{"epicenter": [44.117512, 20.189369], "magnitude": 6}, {"epicenter": [43.886935, 20.556935], "magnitude": 8.5}, {"epicenter": [43.654546, 20.838094], "magnitude": 8}, {"epicenter": [45.415727, 20.205249], "magnitude": 8}]
